Puzzle
----

https://thefiddler.substack.com/p/can-you-weave-the-web

Summary:
Pick 2 point at random in a square and cover all points along the line joining them.
Which points are the most likely to be covered, and by how much relative to the least likely points.

Solution
----

The only feasible scheme I can think of is to create a discrete parallel of NxN points, and calculate these values for that discrete case by exhaustive enumeration of all the cases. And see how it progresses as N increases, and hopefully it converges to something. Let's give it a shot.


In [ ]:
def count_coverage(N, normalize=True, return_counts=False):
    counts = [[0 for _ in range(N)] for _ in range(N)]
    
    #(x1, y1) and (x2, y2) are the randomly selected points.
    # (xt, yt) is a (target) point on the line joining (x1, y1) and (x2, y2).
    for x1 in range(N):
        for x2 in range(N):
            dx = x1 - x2
            for y1 in range(N):
                for y2 in range(N):
                    if (x1, y1) != (x2, y2):                      
                        dy = y1 - y2
                        slope = dy / dx if dx != 0 else float('inf')
                        # Slope low, iterate over x, calculate y
                        # Slope high, iterate over y, calculate x
                        if -1 < slope < 1:
                            for xt in range(N):
                                yt = round(slope * (xt - x1) + y1)
                                if 0 <= yt < N:
                                    counts[xt][yt] += 1                                    
                        else:
                            for yt in range(N):
                                xt = round(x1 + (yt - y1) / slope) if dx != 0 else x1
                                if 0 <= xt < N:
                                    counts[xt][yt] += 1
    
    if (normalize):
        # Normalize the counts to the range [0, 1] to some extent.
        normalization_factor = N * N *(N * N - 1)      
        for i in range(N):
            for j in range(N):
                counts[i][j] /= normalization_factor

    max_value = 0
    min_value = float('inf')
    for i in range(N):
        for j in range(N):
            if counts[i][j] > max_value:
                max_value = counts[i][j]
            if counts[i][j] < min_value:
                min_value = counts[i][j]
    
    max_locations = []
    min_locations = [] 
    for i in range(N):
        for j in range(N):
            if counts[i][j] == max_value:
                max_locations.append("(%.3f, %.3f)"%(i/(N-1), j/(N-1)))
            if counts[i][j] == min_value:
                min_locations.append("(%.3f, %.3f)"%(i/(N-1), j/(N-1)))
    
    max_to_min = max_value / min_value if min_value != 0 else float('inf')

    print(f"N:{N} ratio: {max_to_min:.4f} max: {max_value:.4f} min: {min_value:.4f} max_locations: {max_locations} min_locations: {min_locations}")
    if return_counts:
        return counts

In [3]:
# Simple example for debug.
count_coverage(3)

N:3 ratio: 1.0909 max: 0.3333 min: 0.3056 max_locations: ['(0.000, 0.500)', '(0.500, 0.000)', '(0.500, 0.500)', '(0.500, 1.000)', '(1.000, 0.500)'] min_locations: ['(0.000, 0.000)', '(0.000, 1.000)', '(1.000, 0.000)', '(1.000, 1.000)']


In [4]:
count_coverage(11, normalize=False, return_counts=True)

N:11 ratio: 2.1210 max: 1858.0000 min: 876.0000 max_locations: ['(0.400, 0.400)', '(0.400, 0.600)', '(0.600, 0.400)', '(0.600, 0.600)'] min_locations: ['(0.000, 0.500)', '(0.500, 0.000)', '(0.500, 1.000)', '(1.000, 0.500)']


[[1062, 968, 1018, 908, 992, 876, 992, 908, 1018, 968, 1062],
 [968, 1024, 1176, 1082, 1220, 1084, 1220, 1082, 1176, 1024, 968],
 [1018, 1176, 1410, 1400, 1556, 1452, 1556, 1400, 1410, 1176, 1018],
 [908, 1082, 1400, 1352, 1590, 1412, 1590, 1352, 1400, 1082, 908],
 [992, 1220, 1556, 1590, 1858, 1736, 1858, 1590, 1556, 1220, 992],
 [876, 1084, 1452, 1412, 1736, 1544, 1736, 1412, 1452, 1084, 876],
 [992, 1220, 1556, 1590, 1858, 1736, 1858, 1590, 1556, 1220, 992],
 [908, 1082, 1400, 1352, 1590, 1412, 1590, 1352, 1400, 1082, 908],
 [1018, 1176, 1410, 1400, 1556, 1452, 1556, 1400, 1410, 1176, 1018],
 [968, 1024, 1176, 1082, 1220, 1084, 1220, 1082, 1176, 1024, 968],
 [1062, 968, 1018, 908, 992, 876, 992, 908, 1018, 968, 1062]]

In [5]:
for i in range(3,60,10):
    count_coverage(i)

N:3 ratio: 1.0909 max: 0.3333 min: 0.3056 max_locations: ['(0.000, 0.500)', '(0.500, 0.000)', '(0.500, 0.500)', '(0.500, 1.000)', '(1.000, 0.500)'] min_locations: ['(0.000, 0.000)', '(0.000, 1.000)', '(1.000, 0.000)', '(1.000, 1.000)']
N:13 ratio: 2.2753 max: 0.1141 min: 0.0502 max_locations: ['(0.500, 0.500)'] min_locations: ['(0.000, 0.417)', '(0.000, 0.583)', '(0.417, 0.000)', '(0.417, 1.000)', '(0.583, 0.000)', '(0.583, 1.000)', '(1.000, 0.417)', '(1.000, 0.583)']
N:23 ratio: 2.3519 max: 0.0615 min: 0.0261 max_locations: ['(0.455, 0.455)', '(0.455, 0.545)', '(0.545, 0.455)', '(0.545, 0.545)'] min_locations: ['(0.000, 0.500)', '(0.500, 0.000)', '(0.500, 1.000)', '(1.000, 0.500)']
N:33 ratio: 2.4108 max: 0.0428 min: 0.0177 max_locations: ['(0.500, 0.500)'] min_locations: ['(0.000, 0.531)', '(0.469, 0.000)', '(0.469, 1.000)', '(0.531, 0.000)', '(0.531, 1.000)', '(1.000, 0.531)']
N:43 ratio: 2.4262 max: 0.0324 min: 0.0134 max_locations: ['(0.476, 0.476)', '(0.476, 0.524)', '(0.524, 0.4

It is clear that the max is in the center, min is at the centers of the edges, and the max to min ratio could be converging to something, but not sure what.
Also, from the matrix for N=11, the center of the edges thing seems to be real, not an artifact of the grid approximation.

In [ ]:
#count_coverage(101)

<pre>N:101 ratio: 2.4621 max: 0.0135 min: 0.0055 max_locations: [(0.49504950495049505, 0.49504950495049505)] min_locations: [(0.0, 0.48514851485148514), (0.0, 0.504950495049505)]</pre>
This is also consistent with the results earlier. But still need to check convergence of ratio.

In [ ]:
#count_coverage(121)

<pre>N:121 ratio: 2.4643 max: 0.0113 min: 0.0046 max_locations: ['(0.500, 0.500)'] min_locations: ['(0.492, 0.000)', '(0.508, 0.000)']</pre>


All consistent-ish so far

----

**Epiphany** has been received.

The probability that we want to compute can be written as 

: Integral over all lines passing through the target point  ( probability first point is on the line * probability second point is on the line)

Since each of the probabilities should be proportional to the length of the line, the whole thing should be proportional to 

: Integral over all lines passing through the target point  ( (Length of line)^2 )

Now this integral can be approximated numerically by picking a bunch of lines and calculating the lengths. And since we know the 2 points we need to evaluate it at, it should be much faster ( O(N) ) than the grid counting approach ( O (N^5) ).


In [7]:
# C is center = (0.5, 0.5)
# B is bottom center = (0.5, 0.0)
# We only need to calculate 0 to pi.
# But since our points are centered, the lengths are symmetric, and so it suffices to calculate from 0 to pi/2.
def calc_probability_ratio(N):
    from math import pi, tan
    
    # theta = 0.
    C_Lsq_sum = 1
    B_Lsq_sum = 1

    for i in range(1, N):
        t = (i * pi) / (2 * N) # t = theta
        tt = tan(t)
        B_y = 0.5 * tt   # intersection of line through B with vertical line x=1.
        if B_y <= 1:
            B_Lsq_sum += (0.25 + B_y * B_y)
        else :
            B_x = 1/tt + 0.5 # intersection of line through B with horizontal line y=1.
            B_Lsq_sum += (1 + (B_x - 0.5) * (B_x - 0.5))

        C_y = B_y + 0.5  # intersection of line through C with vertical line x=1.
        if C_y <= 1:
            C_Lsq_sum += (1 + (2 * C_y - 1) * (2 * C_y - 1))
        else:            
            C_x = 0.5/tt + 0.5  # intersection of line through C with horizontal line y=1.
            C_Lsq_sum += (1 + (2 * C_x - 1) * (2 * C_x - 1))
    
    ratio = C_Lsq_sum / B_Lsq_sum
    print(f"Probability ratio for N={N} is {ratio:.5f}  [ = {C_Lsq_sum:.4f} / {B_Lsq_sum:.4f} ]")
    return ratio
        

In [8]:
for i in range(101,9999,1000):
    calc_probability_ratio(i)

Probability ratio for N=101 is 1.98830  [ = 128.5920 / 64.6742 ]
Probability ratio for N=1101 is 1.99893  [ = 1401.8363 / 701.2940 ]
Probability ratio for N=2101 is 1.99944  [ = 2675.0760 / 1337.9132 ]
Probability ratio for N=3101 is 1.99962  [ = 3948.3157 / 1974.5328 ]
Probability ratio for N=4101 is 1.99971  [ = 5221.5552 / 2611.1526 ]
Probability ratio for N=5101 is 1.99977  [ = 6494.7948 / 3247.7724 ]
Probability ratio for N=6101 is 1.99981  [ = 7768.0344 / 3884.3922 ]
Probability ratio for N=7101 is 1.99983  [ = 9041.2739 / 4521.0121 ]
Probability ratio for N=8101 is 1.99985  [ = 10314.5135 / 5157.6318 ]
Probability ratio for N=9101 is 1.99987  [ = 11587.7530 / 5794.2515 ]


Um, these numbers are very different from the grid numbers. This looks like it converging to 2, and that might have been converging to 2.5

----

I am realizing that the hand-wavy probability is proportional to length is the problem.

If we do this properly as an integral in polar coordinates, then the probability will be proportional to the area = radius^2 * dtheta. And radius = length for point B, but radius = length / 2 for point C. So, overall, we'll need to sum something like length^4, not length^2 and there is some constant factor also different for B vs C.

So, let's try again.

In [9]:
# C is center = (0.5, 0.5)
# B is bottom center = (0.5, 0.0)
# We only need to calculate 0 to pi.
# But since our points are centered, the lengths are symmetric, and so it suffices to calculate from 0 to pi/2.
def calc_probability_ratio_take2(N):
    from math import pi, tan, sqrt
    
    C_sum = 0
    B_sum = 0

    for i in range(0, N):
        t = (i * pi) / (2 * N) # t = theta
        tt = tan(t)
        B_y = 0.5 * tt   # intersection of line through B with vertical line x=1.
        if B_y <= 1:
            B_len = sqrt(0.25 + B_y * B_y)
        else :
            B_x = 1/tt + 0.5 # intersection of line through B with horizontal line y=1.
            B_len = sqrt(1 + (B_x - 0.5) * (B_x - 0.5))
        B_area = B_len ** 2 # * dt 
        B_prob = B_area ** 2 # because we are selecting 2 points.

        C_y = B_y + 0.5  # intersection of line through C with vertical line x=1.
        if C_y <= 1:
            C_len = sqrt(1 + (2 * C_y - 1) * (2 * C_y - 1))
        else:            
            C_x = 0.5/tt + 0.5  # intersection of line through C with horizontal line y=1.
            C_len = sqrt(1 + (2 * C_x - 1) * (2 * C_x - 1))
        C_area = ((C_len/2) ** 2) * 2 # * dt 
        C_prob = C_area ** 2 # because we are selecting 2 points.

        B_sum += B_prob

        C_sum += C_prob

    ratio = C_sum / B_sum
    print(f"Probability ratio for N={N} is {ratio:.5f}  [ = {C_sum:.4f} / {B_sum:.4f} ]")
    return ratio

for i in range(101,9999,1000):
    calc_probability_ratio_take2(i)
    

Probability ratio for N=101 is 0.80694  [ = 42.8605 / 53.1149 ]
Probability ratio for N=1101 is 0.80064  [ = 467.2784 / 583.6315 ]
Probability ratio for N=2101 is 0.80034  [ = 891.6918 / 1114.1466 ]
Probability ratio for N=3101 is 0.80023  [ = 1316.1051 / 1644.6627 ]
Probability ratio for N=4101 is 0.80017  [ = 1740.5183 / 2175.1791 ]
Probability ratio for N=5101 is 0.80014  [ = 2164.9315 / 2705.6957 ]
Probability ratio for N=6101 is 0.80012  [ = 2589.3447 / 3236.2123 ]
Probability ratio for N=7101 is 0.80010  [ = 3013.7579 / 3766.7290 ]
Probability ratio for N=8101 is 0.80009  [ = 3438.1711 / 4297.2453 ]
Probability ratio for N=9101 is 0.80008  [ = 3862.5843 / 4827.7616 ]


Well, the center prob is now less than the edge prob ... ???? !!!!

--

I think the issue may be that while our calculations for B and C will both approach the correct integral, they will do so at different rates. And so, the ratio of their probabilities may not be correct.

The differences can be seen by comparing the probability of a nearly vertical line for B and C, which should be the same. But the C value will be d-theta/4, while the B value will be d-theta/2. So, in this direction, B's probability is twice what is should be relative to C's probability.

----

I am going to give it another shot calculating the exact probability, just to make sure I didn't miss something in dropping common factors.


In [10]:
# C is center = (0.5, 0.5)
# B is bottom center = (0.5, 0.0)
# We only need to calculate 0 to pi.
# But since our points are centered, the lengths are symmetric, and so it suffices to calculate from 0 to pi/2.
def calc_probability_ratio_take3(N):
    from math import pi, tan, sqrt
    
    C_sum = 0
    B_sum = 0

    dt = pi / (2 * N)  # delta theta

    for i in range(0, N):
        t = i * dt # t = theta
        tt = tan(t)
        B_y = 0.5 * tt   # intersection of line through B with vertical line x=1.
        if B_y <= 1:
            B_len = sqrt(0.25 + B_y * B_y)
        else :
            B_x = 1/tt + 0.5 # intersection of line through B with horizontal line y=1.
            B_len = sqrt(1 + (B_x - 0.5) * (B_x - 0.5))
        B_area = (B_len ** 2) * dt / 2
        B_prob = B_area ** 2 # because we are selecting 2 points.

        C_y = B_y + 0.5  # intersection of line through C with vertical line x=1.
        if C_y <= 1:
            C_len = sqrt(1 + (2 * C_y - 1) * (2 * C_y - 1))
        else:            
            C_x = 0.5/tt + 0.5  # intersection of line through C with horizontal line y=1.
            C_len = sqrt(1 + (2 * C_x - 1) * (2 * C_x - 1))
        C_area = 2 * ((C_len/2) ** 2) * dt / 2
        C_prob = C_area ** 2 # because we are selecting 2 points.

        B_sum += B_prob

        C_sum += C_prob

    ratio = C_sum / B_sum
    print(f"Probability ratio for N={N} is {ratio:.5f}  [ = {C_sum:.4f} / {B_sum:.4f} ]")
    return ratio

for i in range(101,9999,1000):
    calc_probability_ratio_take3(i)
    

Probability ratio for N=101 is 0.80694  [ = 0.0026 / 0.0032 ]
Probability ratio for N=1101 is 0.80064  [ = 0.0002 / 0.0003 ]
Probability ratio for N=2101 is 0.80034  [ = 0.0001 / 0.0002 ]
Probability ratio for N=3101 is 0.80023  [ = 0.0001 / 0.0001 ]
Probability ratio for N=4101 is 0.80017  [ = 0.0001 / 0.0001 ]
Probability ratio for N=5101 is 0.80014  [ = 0.0001 / 0.0001 ]
Probability ratio for N=6101 is 0.80012  [ = 0.0000 / 0.0001 ]
Probability ratio for N=7101 is 0.80010  [ = 0.0000 / 0.0000 ]
Probability ratio for N=8101 is 0.80009  [ = 0.0000 / 0.0000 ]
Probability ratio for N=9101 is 0.80008  [ = 0.0000 / 0.0000 ]


Okay, that's the same.

-----

Next idea is to use online solvers to calculate these integrals exactly.

Probability of a line through C is

: 2 * [ Integral from 0 to pi/4  (( 1 + tan(x)^2 )/2 dx/2 ) ^2  + Integral from pi/4 to pi/2  (( 1 + cot(x)^2 )/2 dx/2 ) ^2 ]

= 1/2 * [ Integral from 0 to pi/4  (( 1 + tan(x)^2 )/2 dx ) ^2  + Integral from pi/4 to pi/2  (( 1 + cot(x)^2 )/2 dx ) ^2]

That has dx^2, but a single integral, which drives the value to 0.


Probability of a line through B is

: 2 * [ Integral from 0 to atan(2)  (( 1/4 + tan(x)^2/4 ) dx/2 ) ^2   +   Integral from atan(2) to pi/2  (( 1 + cot(x)^2 ) dx/2 ) ^2 ]

= 1/2 * [ Integral from 0 to atan(2)  (( 1 + tan(x)^2 )/4 dx ) ^2   +   Integral from atan(2) to pi/2  (( 1 + cot(x)^2 ) dx ) ^2 ]

Ratio C/B = 

= 1 * [ 
        Integral from 0 to pi/4  (4) dx   +   
        Integral from pi/4 to atan(2) (2 * ( 1 + cot(x)^2 ) / (1 + tan(x)^2) ) ^2 dx   +  
        Integral from atan(2) to pi/2  (1/4) dx 
      ]

where I have cancelled out the second dx.


Acc to Wolfram Alpha 

First Integral = pi = 3.14159

https://www.wolframalpha.com/input?i=Integral+from+0+to+pi%2F4++%284%29+dx

Second Integral = 0.45367

https://www.wolframalpha.com/input?i=Integral+from+pi%2F4+to+atan%282%29+%282+*+%28+1+%2B+cot%28x%29%5E2+%29+%2F+%281+%2B+tan%28x%29%5E2%29+%29+%5E2+dx

Third Integral = 0.11591

https://www.wolframalpha.com/input?i=Integral+from+atan%282%29+to+pi%2F2++%281%2F4%29+dx


So, Ratio C/B = (3.14159 + 0.45367 + 0.11591 ) = 3.71117

Ha, ha. Yet another number, inconsistent with all the rest. :)

I guess the error is taking the ratio by parts.   (A + B) / (C + D) == A/(C+D) + B /(C+D) != A/C + B/D 

---

So, let me do it again, evaluating each separately (ignoring the extra dx as a constant).

Probability of a line through C is

: 2 * [ Integral from 0 to pi/4  (( 1 + tan(x)^2 )/2 dx/2 ) ^2  + Integral from pi/4 to pi/2  (( 1 + cot(x)^2 )/2 dx/2 ) ^2 ]

= 1/2 * [ Integral from 0 to pi/4  (( 1 + tan(x)^2 )/2 dx ) ^2  + Integral from pi/4 to pi/2  (( 1 + cot(x)^2 )/2 dx ) ^2]

= dx/4 [ Integral from 0 to pi/4  ( 1 + tan(x)^2 ) ^2  dx ]

= dx/4 * 4/3 

= dx * 1/3

= dx * 4/12

https://www.wolframalpha.com/input?i=Integral+from+0+to+pi%2F4++%28+1+%2B+tan%28x%29%5E2+%29+%5E2++dx



Probability of a line through B is

: 2 * [ Integral from 0 to atan(2)  (( 1/4 + tan(x)^2/4 ) dx/2 ) ^2   +   Integral from atan(2) to pi/2  (( 1 + cot(x)^2 ) dx/2 ) ^2 ]

= 1/2 * [ Integral from 0 to atan(2)  (( 1 + tan(x)^2 )/4 dx ) ^2   +   Integral from atan(2) to pi/2  (( 1 + cot(x)^2 ) dx ) ^2 ]

= dx/2 * [ 1/16 * Integral from 0 to atan(2)  ( 1 + tan(x)^2 )^2 dx   +   Integral from atan(2) to pi/2  ( 1 + cot(x)^2 )^2 dx]

= dx/2 * [ 1/16 * 14/3 + 13/24 ]

= dx/2 * [ 14/48 + 26 /48 ]

= dx/2 * [ 40/48 ]

= dx * 5/12

https://www.wolframalpha.com/input?i=Integral+from+0+to+atan%282%29++%28+1+%2B+tan%28x%29%5E2+%29%5E2+dx
https://www.wolframalpha.com/input?i=Integral+from+atan%282%29+to+pi%2F2++%28+1+%2B+cot%28x%29%5E2+%29%5E2+dx

Ratio C/B

= 4/5 

= 0.8

Okay, that's consistent with the numerical result in take3.


-----

So, in summary, not sure. I think I'll go with the grid simulation results.